# Problem 3

This problem's purpose is to build a convolutional neural network to classify images as hot dogs or not-hot dogs. This is the same problem as seen in the HBO TV show "Silicon Valley" (https://www.youtube.com/watch?v=pqTntG1RXSY).  We'll be using the dataset put together by a user on Kaggle (https://www.kaggle.com/dansbecker/hot-dog-not-hot-dog) which contains 498 training images and 500 test images.

There are two parts to this assignment:

1. A simple CNN is given below.  Due to the small sample size it has a very poor test set accuracy (around 55\%). Your task is to build a CNN that can beat this test set accuracy by a large margin (better than or equal to 70\% test set accuracy).
2. Describe 3 changes that you made beyond what is given in this notebook and explain what effect they had on the test set accuracy (see below for more instructions).

### Submission

Submit this completed and executed notebook on Quercus that shows your best test set accuracy. We will run a friendly competition in class to see who can achieve the best test set accuracy (for bonus points, bragging rights and a small prize).


# Student Info

###Name: Cole Shulman
###Student Number: 1004021408

# Imports

In [ ]:
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from keras.applications.mobilenet import MobileNet, preprocess_input

from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K


# Loading Hotdog-Not-Hotdog Dataset 

In [ ]:
# Download files
!wget https://briankeng.com/files/hotdog.tar.gz
!tar -xvzf hotdog.tar.gz

--2022-03-11 20:49:39--  https://briankeng.com/files/hotdog.tar.gz
Resolving briankeng.com (briankeng.com)... 192.0.78.240, 192.0.78.156
Connecting to briankeng.com (briankeng.com)|192.0.78.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46732258 (45M) [application/octet-stream]
Saving to: ‘hotdog.tar.gz.2’

hotdog.tar.gz.2     100%[===================>]  44.57M   103MB/s    in 0.4s    

2022-03-11 20:49:40 (103 MB/s) - ‘hotdog.tar.gz.2’ saved [46732258/46732258]

hotdog/
hotdog/test/
hotdog/test/hot_dog/
hotdog/test/hot_dog/324507.jpg
hotdog/test/hot_dog/800992.jpg
hotdog/test/hot_dog/716049.jpg
hotdog/test/hot_dog/588881.jpg
hotdog/test/hot_dog/570799.jpg
hotdog/test/hot_dog/838604.jpg
hotdog/test/hot_dog/315220.jpg
hotdog/test/hot_dog/612440.jpg
hotdog/test/hot_dog/250715.jpg
hotdog/test/hot_dog/292683.jpg
hotdog/test/hot_dog/291354.jpg
hotdog/test/hot_dog/380963.jpg
hotdog/test/hot_dog/533521.jpg
hotdog/test/hot_dog/558890.jpg
hotdog/test/hot_dog/40850

In [ ]:
# Re-scaled dimensions of our images.
img_width, img_height = 299, 299

train_data_dir = 'hotdog/train'
test_data_dir = 'hotdog/test'

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

# Building a CNN From Scratch

In [ ]:
def mymodel():
    model = Sequential()
    model.add(Conv2D(16, kernel_size=(3, 3),
                 strides=1, padding='valid',
                 activation='relu',
                 input_shape=input_shape))
    model.add(Conv2D(16, kernel_size=(3, 3),
                 strides=2, padding='valid',
                 activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(1024,Activation('relu')))
    model.add(Dropout(0.7))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
               optimizer=keras.optimizers.Adam(lr=0.1),
               metrics=['accuracy'])
    
    return model

# Test function
mymodel().summary()

Model: "sequential_97"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1852 (Conv2D)        (None, 297, 297, 16)      448       
                                                                 
 conv2d_1853 (Conv2D)        (None, 148, 148, 16)      2320      
                                                                 
 max_pooling2d_42 (MaxPoolin  (None, 74, 74, 16)       0         
 g2D)                                                            
                                                                 
 flatten_78 (Flatten)        (None, 87616)             0         
                                                                 
 dense_179 (Dense)           (None, 1024)              89719808  
                                                                 
 dropout_86 (Dropout)        (None, 1024)              0         
                                                     

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
# You may optionally change these parameters
batch_size = 50
epochs = 10

# Data parameters (DO NOT MODIFY)
num_train_samples = 498
num_test_samples = 500

# Data generators (DO NOT MODIFY)
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [ ]:
def evaluate_model(runs=5):
    ''' DO NOT MODIFY THIS FUNCTION '''
    scores = [] 
    for i in range(runs):
        print('Executing run %d' % (i+1))
        model = mymodel()
        model.fit_generator(train_generator,
                            callbacks=[],
                            steps_per_epoch=num_train_samples // batch_size,
                            epochs=epochs, verbose=0)
        print(' * Evaluating model on test set')
        scores.append(model.evaluate_generator(test_generator, 
                                               steps=num_test_samples // batch_size,
                                               verbose=0))
        print(' * Test set Loss: %.4f, Accuracy: %.4f' % (scores[-1][0], scores[-1][1]))
        
    accuracies = [score[1] for score in scores]     
    return np.mean(accuracies), np.std(accuracies)
        
mean_accuracy, std_accuracy = evaluate_model(runs=5)

Executing run 1


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  # Remove the CWD from sys.path while we load stuff.


 * Evaluating model on test set


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  


 * Test set Loss: 0.7048, Accuracy: 0.4980
Executing run 2
 * Evaluating model on test set
 * Test set Loss: 0.6933, Accuracy: 0.5000
Executing run 3
 * Evaluating model on test set
 * Test set Loss: 0.6938, Accuracy: 0.5000
Executing run 4
 * Evaluating model on test set
 * Test set Loss: 0.6932, Accuracy: 0.5000
Executing run 5
 * Evaluating model on test set
 * Test set Loss: 0.6932, Accuracy: 0.5000


In [ ]:
 # You will be evaluated on your mean test set accuracy over 5 runs
print('Mean test set accuracy over 5 runs: %.4f +/- %.4f' % (mean_accuracy, std_accuracy))

Mean test set accuracy over 5 runs: 0.4996 +/- 0.0008


# Unfrozen Layers in a Pre-Trained Model

In [ ]:
incres_base= InceptionResNetV2(include_top=False, weights="imagenet")
incres_base

219070464/219055592 [==============================] - 1s 0us/step


In [ ]:
from keras.layers.pooling import GlobalAveragePooling2D
def mymodel():
    model = Sequential()
    model.add(incres_base)
    model.add(GlobalAveragePooling2D())
    model.add(Flatten())
    model.add(Dense(1024,Activation('relu')))
    model.add(Dropout(0.7))
    model.add(Dense(1, activation='sigmoid'))
    # Freeze layers in the base model (i.e. only train the classifier)
    for layer in incres_base.layers:
      layer.trainable = True

    model.compile(loss='binary_crossentropy',
               optimizer=keras.optimizers.Adam(lr=0.1),
               metrics=['accuracy'])
    
    return model

# Test function
mymodel().summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, None, None, 1536)  54336736 
 onal)                                                           
                                                                 
 global_average_pooling2d (G  (None, 1536)             0         
 lobalAveragePooling2D)                                          
                                                                 
 flatten (Flatten)           (None, 1536)              0         
                                                                 
 dense (Dense)               (None, 1024)              1573888   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 1)                 1

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
# You may optionally change these parameters
batch_size = 50
epochs = 3
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Data parameters (DO NOT MODIFY)
num_train_samples = 498
num_test_samples = 500

# Data generators (DO NOT MODIFY)
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [ ]:
def evaluate_model(runs=5):
    ''' DO NOT MODIFY THIS FUNCTION '''
    scores = [] 
    for i in range(runs):
        print('Executing run %d' % (i+1))
        model = mymodel()
        model.fit_generator(train_generator,
                            callbacks=[],
                            steps_per_epoch=num_train_samples // batch_size,
                            epochs=epochs, verbose=0)
        print(' * Evaluating model on test set')
        scores.append(model.evaluate_generator(test_generator, 
                                               steps=num_test_samples // batch_size,
                                               verbose=0))
        print(' * Test set Loss: %.4f, Accuracy: %.4f' % (scores[-1][0], scores[-1][1]))
        
    accuracies = [score[1] for score in scores]     
    return np.mean(accuracies), np.std(accuracies)
        
mean_accuracy, std_accuracy = evaluate_model(runs=5)

Executing run 1


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
 # You will be evaluated on your mean test set accuracy over 5 runs
print('Mean test set accuracy over 5 runs: %.4f +/- %.4f' % (mean_accuracy, std_accuracy))

Mean test set accuracy over 5 runs: 0.9236 +/- 0.0118


# Selecting Specific Unfozen Layers

In [ ]:
incres_base= InceptionResNetV2(include_top=False, weights="imagenet")
incres_base

In [ ]:
from keras.layers.pooling import GlobalAveragePooling2D
def mymodel():
    model = Sequential()
    model.add(incres_base)
    model.add(GlobalAveragePooling2D())
    model.add(Flatten())
    model.add(Dense(1024,Activation('relu')))
    model.add(Dropout(0.7))
    model.add(Dense(1, activation='sigmoid'))
    # Freeze layers in the base model (i.e. only train the classifier)
    for layer in incres_base.layers:
      if layer.name in ["conv2d_199","conv2d_201","conv2d_202","block8_10_conv","conv_7b"]:
        layer.trainable = True

    model.compile(loss='binary_crossentropy',
               optimizer=keras.optimizers.Adam(lr=0.1),
               metrics=['accuracy'])
    
    return model

# Test function
mymodel().summary()

the names are working
the names are working
the names are working
the names are working
the names are working
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, None, None, 1536)  54336736 
 onal)                                                           
                                                                 
 global_average_pooling2d (G  (None, 1536)             0         
 lobalAveragePooling2D)                                          
                                                                 
 flatten (Flatten)           (None, 1536)              0         
                                                                 
 dense (Dense)               (None, 1024)              1573888   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
            

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
# You may optionally change these parameters
batch_size = 50
epochs = 3
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Data parameters (DO NOT MODIFY)
num_train_samples = 498
num_test_samples = 500

# Data generators (DO NOT MODIFY)
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [ ]:
def evaluate_model(runs=5):
    ''' DO NOT MODIFY THIS FUNCTION '''
    scores = [] 
    for i in range(runs):
        print('Executing run %d' % (i+1))
        model = mymodel()
        model.fit_generator(train_generator,
                            callbacks=[],
                            steps_per_epoch=num_train_samples // batch_size,
                            epochs=epochs, verbose=0)
        print(' * Evaluating model on test set')
        scores.append(model.evaluate_generator(test_generator, 
                                               steps=num_test_samples // batch_size,
                                               verbose=0))
        print(' * Test set Loss: %.4f, Accuracy: %.4f' % (scores[-1][0], scores[-1][1]))
        
    accuracies = [score[1] for score in scores]     
    return np.mean(accuracies), np.std(accuracies)
        
mean_accuracy, std_accuracy = evaluate_model(runs=5)

Executing run 1
the names are working
the names are working
the names are working
the names are working
the names are working


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  # Remove the CWD from sys.path while we load stuff.


 * Evaluating model on test set


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  


 * Test set Loss: nan, Accuracy: 0.5000
Executing run 2
the names are working
the names are working
the names are working
the names are working
the names are working


KeyboardInterrupt: ignored

In [ ]:
 # You will be evaluated on your mean test set accuracy over 5 runs
print('Mean test set accuracy over 5 runs: %.4f +/- %.4f' % (mean_accuracy, std_accuracy))

# Frozen Layers in a Pre-Trained Model

In [ ]:
incres_base= InceptionResNetV2(include_top=False, weights="imagenet")
incres_base

219070464/219055592 [==============================] - 2s 0us/step


In [ ]:
incres_base.summary()

Model: "inception_resnet_v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d (Conv2D)                (None, None, None,   864         ['input_1[0][0]']                
                                32)                                                               
                                                                                                  
 batch_normalization (BatchNorm  (None, None, None,   96         ['conv2d[0][0]']                 
 alization)                     32)                                             

In [ ]:
from keras.layers.pooling import GlobalAveragePooling2D
def mymodel():
    model = Sequential()
    model.add(incres_base)
    model.add(GlobalAveragePooling2D())
    model.add(Flatten())
    model.add(Dense(1024,Activation('relu')))
    model.add(Dropout(0.7))
    model.add(Dense(1, activation='sigmoid'))
    # Freeze layers in the base model (i.e. only train the classifier)
    for layer in incres_base.layers:
      layer.trainable = False

    model.compile(loss='binary_crossentropy',
               optimizer=keras.optimizers.Adam(lr=0.1),
               metrics=['accuracy'])
    
    return model

# Test function
mymodel().summary()

Model: "sequential_89"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, None, None, 1536)  54336736 
 onal)                                                           
                                                                 
 global_average_pooling2d_76  (None, 1536)             0         
  (GlobalAveragePooling2D)                                       
                                                                 
 flatten_70 (Flatten)        (None, 1536)              0         
                                                                 
 dense_163 (Dense)           (None, 1024)              1573888   
                                                                 
 dropout_78 (Dropout)        (None, 1024)              0         
                                                                 
 dense_164 (Dense)           (None, 1)               

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


### Loading data on the fly

We load the data directly from the images on disk via these Keras helper functions (`ImageDataGenerator` and `flow_from_directory`). It performs two transformations: 

* Rescaling pixels to be between [0, 1]
* Resizing images to be in `img_width`x`img_height` (150x150)

During training for each batch, the images are read from disk on the fly, loaded into memory and then the transformations are applied.

In [ ]:
# You may optionally change these parameters
batch_size = 50
epochs = 10
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Data parameters (DO NOT MODIFY)
num_train_samples = 498
num_test_samples = 500

# Data generators (DO NOT MODIFY)
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [ ]:
def evaluate_model(runs=5):
    ''' DO NOT MODIFY THIS FUNCTION '''
    scores = [] 
    for i in range(runs):
        print('Executing run %d' % (i+1))
        model = mymodel()
        model.fit_generator(train_generator,
                            callbacks=[],
                            steps_per_epoch=num_train_samples // batch_size,
                            epochs=epochs, verbose=0)
        print(' * Evaluating model on test set')
        scores.append(model.evaluate_generator(test_generator, 
                                               steps=num_test_samples // batch_size,
                                               verbose=0))
        print(' * Test set Loss: %.4f, Accuracy: %.4f' % (scores[-1][0], scores[-1][1]))
        
    accuracies = [score[1] for score in scores]     
    return np.mean(accuracies), np.std(accuracies)
        
mean_accuracy, std_accuracy = evaluate_model(runs=5)

Executing run 1


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  # Remove the CWD from sys.path while we load stuff.


 * Evaluating model on test set


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  


 * Test set Loss: 0.1957, Accuracy: 0.9100
Executing run 2
 * Evaluating model on test set
 * Test set Loss: 0.1690, Accuracy: 0.9100
Executing run 3
 * Evaluating model on test set
 * Test set Loss: 0.1893, Accuracy: 0.9260
Executing run 4
 * Evaluating model on test set
 * Test set Loss: 0.5968, Accuracy: 0.9160
Executing run 5
 * Evaluating model on test set
 * Test set Loss: 0.8385, Accuracy: 0.9160


In [ ]:
 # You will be evaluated on your mean test set accuracy over 5 runs
print('Mean test set accuracy over 5 runs: %.4f +/- %.4f' % (mean_accuracy, std_accuracy))

Mean test set accuracy over 5 runs: 0.9156 +/- 0.0059
